In [ ]:
# import the library
from pynq import Overlay     # import the overlay
from pynq import allocate    # import for CMA (contingeous memory allocation)
from pynq import DefaultIP   # import the ip connector library for extension
from pynq import Interrupt
import pynq_driver
import asyncio
import numpy as np
import os
import subprocess
import re
import time

In [ ]:
overlay = Overlay("system.bit")
help(overlay)

In [ ]:

overlay.interrupt_pins

In [ ]:
# create an instance of the interrupt
my_interrupt = Interrupt()

In [ ]:
# Load input from .npy file
input_array  = np.load("inputGenbit.npy").astype(np.float32)  # shape (20,4,4,1)
output_array = np.zeros(input_array.shape, dtype=np.float32)

# Allocate physically contiguous memory for input and output
input_buffer = allocate(shape=input_array.shape, dtype=np.float32)
output_buffer = allocate(shape=output_array.shape, dtype=np.float32)

In [ ]:
# check input shape
print(f"input array shape {input_array.shape}")
print(f"output array shape {output_array.shape}")

input array shape (10000, 4, 4, 1)
output array shape (10000, 4, 4, 1)


In [ ]:
# copy data to input buffer
np.copyto(input_buffer, input_array)
input_buffer.flush()

In [ ]:
# get the ip and initialize the system
ip = overlay.myproject_gem_1  # Replace with your IP instance name
ip.set_input (0, input_buffer)
ip.set_output(0, output_buffer)
ip.set_amt_query(input_array.shape[0])
ip.prepare_intr()

In [ ]:
async def wait_for_acc():
    print("starting the accelerator")
    ip.ctrl_start()
    print("waiting for the accelerator to finish")
    await my_interrupt.wait()
    print("accelerator has finished")


In [ ]:
#### get event loop from asyncio
loop = asyncio.get_event_loop()

In [ ]:
task = loop.create_task(wait_for_acc())
loop.run_until_complete(task)

In [14]:
output_buffer.invalidate()

In [15]:
print(output_buffer)

[[[[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.49609375]
   [0.49609375]
   [0.49609375]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5029297 ]]]


 [[[0.5       ]
   [0.5       ]
   [0.49609375]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]

  [[0.49609375]
   [0.49609375]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]]


 [[[0.5       ]
   [0.48828125]
   [0.49609375]
   [0.49609375]]

  [[0.5       ]
   [0.48828125]
   [0.5       ]
   [0.5       ]]

  [[0.49609375]
   [0.49609375]
   [0.49609375]
   [0.5       ]]

  [[0.5       ]
   [0.5       ]
   [0.5       ]
   [0.5       ]]]


 ...


 [[[0.5       ]
   [0.49609375]
   [0.5       ]
   [0.5       ]]

  [[0.5       ]
   [0.49609375]
   [0.5       ]
   [0.5029297 ]]

  [[0.5       ]
   [0.49609375]
   [0.5       ]
   [0.5029297 ]

In [ ]:
# convert it to numpy array
print("we got output shape:", output_buffer.shape)
outNp = np.array(output_buffer)

In [ ]:
# save it to .npy file
np.save("out_hw.npy", outNp)